In [3]:
# Install the libraries
import numpy as np
from skimage import io, img_as_ubyte
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import matplotlib.pyplot as plt

######## version 1 ########
# Image path 
noisy_img_path = "/Users/jiyoung/Desktop/univ/'23-2/Engineering Design/workspace/dog_noisy.png"
clean_img_path = "/Users/jiyoung/Desktop/univ/'23-2/Engineering Design/workspace/dog2.png"

# Read the image
noisy_img = io.imread(noisy_img_path).astype(np.float32) / 255.0
clean_img = io.imread(clean_img_path).astype(np.float32) / 255.0
######## version 1 ########

# ######## version 2 ########
# noisy_img = io.imread("/Users/jiyoung/Desktop/univ/'23-2/Engineering Design/workspace/dog_noisy.png").astype(np.float32) / 255.0
# clean_img = io.imread("/Users/jiyoung/Desktop/univ/'23-2/Engineering Design/workspace/dog.png").astype(np.float32) / 255.0
# ######## version 2 ########

In [2]:
def NLMeans_Filter(noisy_img, h=0.1, patch_size=5, search_window=11):
    denoised_img = np.zeros_like(noisy_img)
    padded_img = np.pad(noisy_img, ((patch_size//2,),(patch_size//2,),(0,)), 'reflect')
    height, width, channels = noisy_img.shape

    patches = np.array([
        padded_img[i:i + patch_size, j:j + patch_size, :]
        for i in range(height)
        for j in range(width)
    ])

    for y in range(height):
        for x in range(width):
            patch = patches[y * width + x];  weight_sum = 0.0;  weighted_sum = np.zeros((channels,))

            for dy in range(-search_window//2, search_window//2 + 1):
                for dx in range(-search_window//2, search_window//2 + 1):
                    yy, xx = y + dy, x + dx
                    if 0 <= yy < height and 0 <= xx < width:
                        neighbor_patch = patches[yy * width + xx];  patch_diff = np.sum((neighbor_patch - patch) ** 2)
                        weight = np.exp(-patch_diff / (h ** 2));    weighted_sum += weight * neighbor_patch[patch_size//2, patch_size//2, :]
                        weight_sum += weight

            denoised_img[y, x, :] = weighted_sum / weight_sum

    return denoised_img

    
denoised_img = NLMeans_Filter(noisy_img, h=0.1, patch_size=5, search_window=11)
# # 이미지 크기 확인
# height, width, _ = noisy_img.shape
# if height < 5 or width < 5:
#     print("이미지 크기가 너무 작아 NLMeans 필터 적용이 불가능합니다.")
# else:
#     # NLMeans 필터 적용
#     denoised_img = non_local_means_denoising(noisy_img, h=0.1, patch_size=5, search_window=11)

#     # 결과 이미지를 화면에 표시
#     plt.figure(figsize=(10, 5))
#     plt.subplot(1, 3, 1)
#     plt.imshow(noisy_img)
#     plt.title("Noisy Image")

#     plt.subplot(1, 3, 2)
#     plt.imshow(denoised_img)
#     plt.title("Denoised Image")

#     plt.subplot(1, 3, 3)
#     plt.imshow(clean_img)
#     plt.title("Clean Image")

# PSNR 계산
psnr = peak_signal_noise_ratio(clean_img, denoised_img)
print("PSNR(non local mean):", psnr)

plt.show()


In [4]:
def NLMeans_Filter(noisy_img, h=0.1, patch_size=5, search_window=11):
    denoised_img = np.zeros_like(noisy_img)
    padded_img = np.pad(noisy_img, ((patch_size//2,),(patch_size//2,),(0,)), 'reflect')
    height, width, channels = noisy_img.shape

    # 패치 사전 계산
    patches = np.array([
        padded_img[i:i + patch_size, j:j + patch_size, :]
        for i in range(height)
        for j in range(width)
    ])

    # 메인 계산 루프
    for y in range(height):
        for x in range(width):
            patch = patches[y * width + x]

            weight_sum = 0.0
            weighted_sum = np.zeros((channels,))

            # 검색 윈도우 처리
            for dy in range(-search_window//2, search_window//2 + 1):
                for dx in range(-search_window//2, search_window//2 + 1):
                    yy, xx = y + dy, x + dx
                    if 0 <= yy < height and 0 <= xx < width:
                        neighbor_patch = patches[yy * width + xx]
                        patch_diff = np.sum((neighbor_patch - patch) ** 2)
                        weight = np.exp(-patch_diff / (h ** 2))
                        weighted_sum += weight * neighbor_patch[patch_size//2, patch_size//2, :]
                        weight_sum += weight

            denoised_img[y, x, :] = weighted_sum / weight_sum

    return denoised_img

    
denoised_img = NLMeans_Filter(noisy_img, h=0.1, patch_size=5, search_window=11)

# PSNR 계산
psnr = peak_signal_noise_ratio(clean_img, denoised_img)
print("PSNR(non local mean):", psnr)

plt.show()

PSNR(non local mean): 15.480759914469605
